### Data cleaning

Este notebook sirve para limpiar la base de datos extraida y almcenada con título `scraped_data.csv`. Aquí se llevarán a cabo métodos de inspección del dataframe para comprobar que los datatypes sean correctos y que la información contenida sea útil y legible en otras herramientas como pueden ser SQL y PowerBI.

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
df = pd.read_csv('../data/scraped_data.csv')
df.head()

,date,lat_and_long,GTOA_Protocol,Boat_Type,Boat_Length,Towing_Inflatable,Trailing_Fishing_Lure,Physical_Contact_With_Boat,Number_of_Adult_Orcas,Number_of_Juvenile_Orcas,Number_of_Uncertain_Age_Orcas,Rudder,Motoring_or_Sailing,Speed_Knots,Sea_State,Wind_Speed_Beaufort,Daylight_or_Darkness,Cloud_Cover,Distance_Off_Land_NM,Depth_Meters,Depth_Gauge,Autopilot,Hull_Topsides_Color,Antifoul_Color,Boat_Damaged,Tow_Required,Crew_Response,Orcas_Behaviour
0,2023-11-01 22:15:00,"32 47.4980 N, 9 54.3980 W","No, did not follow protocol, interaction laste...",Sail,10 - 12.5m,Not towing,No,No,1,0,0,Spade,Sailing,5 - 7,Moderate,5 - 6 (17 - 27 knots),Night,0 - 25%,Over 10,200m+,On,On,White/light,Blue,No,No,"Orca interaction at 10:15pm on 01/11, 40 miles...",I would describe the behaviour of the Orca dur...
1,2023-10-31 07:50:00,"39 26.0000 N, 9 23.0000 W","Yes, followed protocol, interaction lasted les...",Sail,12.5 - 15m,Not towing,No,Yes,2,5,0,Twin rudder,Motoring,5 - 7,Rough,3 - 4 (7 - 16 knots),Day,50 - 75%,2 - 5,40 - 200m,On,On,White/light,Black,"Yes, moderate - immediate repairs required",No,We had sandbags on our sugar scoops and metal ...,Juveniles hitting the rudders adults close by
2,2023-09-19 11:00:00,"37 40.0000 N, 8 54.0000 W","No, did not follow protocol, interaction laste...",Sail,12.5 - 15m,Not towing,No,Yes,1,0,0,Spade,Motoring,5 - 7,Calm,3 - 4 (7 - 16 knots),Day,0 - 25%,2 - 5,40 - 200m,On,On,White/light,Other,"Yes, moderate - immediate repairs required",No,We saw the orca approach from 10 o’clock posit...,There was an initial approach 45 minutes earli...
3,2023-09-01 13:15:00,"45 36.0000 N, 3 45.0000 W","Yes, followed protocol, interaction lasted 10 ...",Sail,Over 15m,Not towing,Yes,Yes,1,2,0,Spade,Sailing,3 - 4,Calm,3 - 4 (7 - 16 knots),Day,25 - 50%,Over 10,200m+,Off,Off,White/light,Black,"Yes, moderate - immediate repairs required",No,Les trois orques passent constamment de bâbord...,Pas de comportement visblement agressif./// No...
4,2023-09-02 03:45:00,"42 45.0000 N, 9 14.0000 W","Yes, followed protocol, interaction lasted les...",Sail,12.5 - 15m,Not towing,No,Yes,1,2,0,Spade,Motorsailing,5 - 7,Calm,0 - 2 (0 - 6 knots),Night,0 - 25%,5 - 10,40 - 200m,On,On,White/light,Black,"Yes, moderate - immediate repairs required",Yes,Arrêt du pilote automatique a la 2 eme interac...,Approche furtive à la première interaction dir...


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154 entries, 0 to 153
Data columns (total 28 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   date                           154 non-null    object
 1   lat_and_long                   154 non-null    object
 2   GTOA_Protocol                  154 non-null    object
 3   Boat_Type                      154 non-null    object
 4   Boat_Length                    154 non-null    object
 5   Towing_Inflatable              154 non-null    object
 6   Trailing_Fishing_Lure          154 non-null    object
 7   Physical_Contact_With_Boat     154 non-null    object
 8   Number_of_Adult_Orcas          154 non-null    object
 9   Number_of_Juvenile_Orcas       154 non-null    object
 10  Number_of_Uncertain_Age_Orcas  154 non-null    object
 11  Rudder                         154 non-null    object
 12  Motoring_or_Sailing            154 non-null    object
 13  Speed

Tenemos 28 columnas, todas de tipo objeto por como se ha escrapeado la información en el notebook `1.WebScraping`. Entre otras cosas se va a hacer lo siguiente:

* Borrar aquellas filas que no contengan información o que estén repetidas
* Corregir cualquier fila de datos que pueda estar en columnas equivocadas
* Separar datos en dos columnas si fuera necesario, como por ejemplo con latitud y longitud que actualmente se encuentra en una sola columna
* Cambiar los datatypes de las columnas
* Quitar duplicados en caso de que los hubiera
* Ya que la mayor parte de las respuestas del formulario que llevaron a cabo los patrones de las embarcaciones que sufrieron una interacción con una orca constaban de varias opciones (y no respuesta libre), será util convertir las variables categóricas en numéricas, vía aplicación de One-Hot encoding. Esto supondrá un paso clave de cara a aplicar métodos de modelos predictivos en el futuro.
* Identificar y rechazar outliers en caso de que los hubiera y siempre razonando si interesa deshacernos de ellos o nos aporta una información útil.
* Comprobación de consistencia de la BBDD.

#### 1. Descripción de las columnas

A continuación se incluye una descripción de las columnas con las que cuenta originalmente nuestra base de datos:

* **date**: Fecha y hora aproximada de la interacción
* **lat_and_long**: Latitud y longitud donde se dio la interacción
* GT**OA_Protocol: Se siguió el **protocolo** de GT Orca Atlántica: Arriar las velas, detener la embarcación, apagar el motor y mantener un perfil bajo así como **duración** de la interacción
* **Boat_Type**: Tipo de barco - Velero | Motor | Barco de pesca
* **Boat_Length**: Eslora del barco (m) - menos de 10m | 10-12.5 | 12.5-15m | Más de 15m
* **Towing_Inflatable**: Se encontraba el barco remolcando una lancha neumática?
* **Trailing_Fishing_Lure**: Se encontraba el barco arrastrando un señuelo de pesca?
* **Physical_Contact_With_Boat**: Hubo contacto físico de las orcas con la embarcación?
* **Number_of_Adult_Orcas**: Número de orcas adultas?
* **Number_of_Juvenile_Orcas**: Número de orcas juveniles?
* **Number_of_Uncertain_Age_Orcas**: Número de orcas de edad incierta?
* **Rudder**: Tipo de timón - Spade/pala | Semi skeg/Semicompensado | Full skeg/Completo | Twin rudder/Doble timón | Keel hung/Quilla corrida
* **Motoring_or_Sailing**: Motor o vela - Vela | Motor | Motor/Vela | Hove-to
* **Speed_Knots**: Velocidad (kts)
* **Sea_State**: Estado del mar - Calma | Moderado | Gruesa
* **Wind_Speed_Beaufort**: Velocidad del viento (Escala de Beaufort) - 0.2 | 3-4 | 5-6 | 7+
* **Daylight_or_Darkness**: Noche/Día - Amanecer | Día | Atardecer | Noche
* **Cloud_Cover**: Cobertura de nubes - 0-25% | 25-50% | 50-75% | 75-100%
* **Distance_Off_Land_NM**: Distancia a tierra (nm) - 0-2 | 2-5 | 5-10 | Más de 10
* **Depth_Meters**: Profundidad (m) - hasta 20m | 20-40m | 40-200m | 200m+
* **Depth_Gauge**: Medidor de profundidad - On | Off
* **Autopilot**: Piloto automático - On | Off
* **Hull_Topsides_Color**: Color del casco - Blanco | Oscuro
* **Antifoul_Color**: Color del antifoul - Negro | Azul | Rojo | Blanco | Verde | Coppercoat | Otro
* **Boat_Damaged**: Fue dañado el barco o necesita reparación alguna? Sí, menor | Sí, moderado | Sí, grandes reparaciones | No
* **Tow_Required**: ¿Fue remolcado? - Yes | No
* **Crew_Response**: Descripción abierta de la interacción así como acciones que se tomaron y su estas  disuadieron o no la interacción con las orcas.
* **Orcas_Behaviour**: Descripción del comportamiento de la/s orca/s

In [9]:
df.GTOA_Protocol.value_counts()

GTOA_Protocol
No, did not follow protocol, interaction lasted less than 10 minutes    40
Yes, followed protocol, interaction lasted 10 to 30 minutes             26
Yes, followed protocol, interaction lasted less than 10 minutes         24
No, did not follow protocol, interaction lasted 10 to 30 minutes        24
Yes, followed protocol, interaction lasted 30 to 60 minutes             15
Yes, followed protocol, interaction lasted more than 60 minutes         11
No, did not follow protocol, interaction lasted 30 to 60 minutes         9
No, did not follow protocol, interaction lasted more than 60 minutes     3
Sail                                                                     1
Unknown                                                                  1
Name: count, dtype: int64